In [17]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns

import train
import models

%load_ext autoreload
%autoreload 2

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
path = "../"
category = "real_data"
files = glob2.glob(f'{path}{category}/*.h5')
files = [f[len(f"'{path}{category}"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [19]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()
epochs = 10
batch_size = 128 
pca_size = 50
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
print(files)
nb_genes =2000

['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'Quake_10x_Spleen', 'Young', 'mouse_ES_cell', 'Adam', 'Quake_10x_Bladder', 'Quake_Smart-seq2_Lung', 'Quake_10x_Limb_Muscle', 'worm_neuron_cell', 'mouse_bladder_cell', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro', '10X_PBMC']


In [20]:
# sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [21]:
# optimal = pd.read_pickle(f"{path}output/pickle_results/real_data/optimal_input_size.pkl")
# optimal.head()

In [22]:
results = pd.DataFrame()
model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu
for dataset in files:
    print(f">> {dataset}")

    data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    Y = np.array(data_mat['Y'])
    X = np.array(data_mat['X'])

    genes_idx, cells_idx = train.filter_data(X, highly_genes=nb_genes)
    X = X[cells_idx][:, genes_idx]
    Y = Y[cells_idx]
    n_clusters = len(np.unique(Y))
    t0 = time.time()
    graph = train.make_graph(
        X,
        Y,
        dense_dim=pca_size,
        node_features=node_features,
        normalize_weights=normalize_weights,
    )

    labels = graph.ndata["label"]
    train_ids = np.where(labels != -1)[0]

    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

    dataloader = dgl.dataloading.NodeDataLoader(
        graph,
        train_ids,
        sampler,
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=1,
    )
    print(
        f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
    )
    t1 = time.time()
    
    for lr in [ 1e-3, 1e-4, 0.5*1e-5,1e-5, 1e-6
              ]:

        for run in range(3):
            t_start = time.time()
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)

            model = models.GCNAE(
                in_feats=pca_size,
                n_hidden=hidden_dim,
                n_layers=n_layers,
                activation=activation,
                dropout=0.1,
                hidden=hidden,
                hidden_relu=hidden_relu,
                hidden_bn=hidden_bn,
            ).to(device)
            if run == 0:
                print(f">", model)

            optim = torch.optim.Adam(model.parameters(), lr=lr)

            scores = train.train(model, optim, epochs, dataloader, n_clusters,  plot=False,
                                cluster=["KMeans", "Leiden"])
            scores["dataset"] = dataset
            scores["run"] = run
            scores["nb_genes"] = nb_genes
            scores["hidden"] = str(hidden)
            scores["hidden_dim"] = str(hidden_dim)
            scores["lr"] = lr

            results = results.append(scores, ignore_index = True)

            results.to_pickle(
                f"../output/pickle_results/{category}/{category}_lr.pkl")
            print("Done")

>> Quake_Smart-seq2_Trachea


/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, [300], False, True
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7f8b4ba935f0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.0235, 0.6314987540245056
Done



ARI 0.0118, 0.7846592664718628
Done


...


ARI 0.7173, 0.46485814452171326
Done



ARI 0.7244, 0.47059884667396545
Done
> GCNAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): WeightedGraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7f8b4ba935f0>)
  (decoder): InnerProductDecoder()
  (encoder): Sequential(
    (0): Linear(in_features=200, out_features=300, bias=True)
  )
)



ARI 0.634, 0.4684520661830902
Done



ARI 0.7106, 0.4876680076122284
Done



ARI 0.6314, 0.4649181663990021
Done


In [23]:
results.groupby("lr").mean()

,ae_end,kmeans_ari,kmeans_cal,kmeans_nmi,kmeans_sil,kmeans_time,leiden_ari,leiden_cal,leiden_nmi,leiden_sil,leiden_time,nb_genes,run
lr,,,,,,,,,,,,,
0.000001,1.614905e+09,0.757427,2244.298072,0.802560,0.481843,7.690216,0.513667,1334.692573,0.720020,0.292956,14.973253,3000.0,1.0
0.000005,1.614904e+09,0.768191,2055.254216,0.808367,0.481235,7.697033,0.516304,1247.045785,0.721120,0.294487,14.675473,3000.0,1.0
0.000010,1.614904e+09,0.778107,1763.814248,0.816500,0.476133,7.701662,0.521487,1093.629245,0.723849,0.300331,14.903587,3000.0,1.0
0.000100,1.614904e+09,0.550136,548.529579,0.675522,0.325868,7.757632,0.516502,335.948688,0.713267,0.228678,15.531728,3000.0,1.0
0.001000,1.614904e+09,0.124347,235.186843,0.241622,0.368898,7.982750,0.496162,80.008492,0.688633,0.030062,16.972422,3000.0,1.0
